In [3]:
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

torch.manual_seed(190401099)


#multilayer perceptronumuz
class MLP(nn.Module):
    def __init__(self, input_size):
        super(MLP, self).__init__()
        self.hiddenLayer1 = nn.Linear(input_size, 100)
        self.relu1 = nn.ReLU()
        self.hiddenLayer2 = nn.Linear(100, 50)
        self.relu2 = nn.ReLU()
        self.outputLayer = nn.Linear(50, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.hiddenLayer1(x)
        out = self.relu1(out)
        out = self.hiddenLayer2(out)
        out = self.relu2(out)
        out = self.outputLayer(out)
        out = self.sigmoid(out)
        return out
    
#veri setlerinden oznitelik ve sinifi almak icin kodumuz
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.x = self.data.iloc[:, :-1].values
        self.y = self.data.iloc[:, -1].values.reshape(-1, 1)

    def __getitem__(self, idx):
        return torch.Tensor(self.x[idx]), torch.Tensor(self.y[idx])

    def __len__(self):
        return len(self.data)


#verileri okuyoruz
train_dataset = CustomDataset('cure_the_princess_train.csv')
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

test_dataset = CustomDataset('cure_the_princess_test.csv')
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

valid_dataset = CustomDataset('cure_the_princess_validation.csv')
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)

#verilere gore modelimizi olusturuyoruz
model = MLP(input_size=13)
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
min_validation_loss = 1

for epoch in range(20):
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        #print(inputs)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation
    total_val_loss = 0.0
    for i, data in enumerate(valid_loader):
        inputs, labels = data
        outputs = model(inputs)
        val_loss = criterion(outputs, labels)
        total_val_loss += val_loss.item()
    if( (total_val_loss/len(valid_loader)) < min_validation_loss ):
    #   print("is good")
        min_validation_loss = (total_val_loss/len(valid_loader))
        torch.save(model.state_dict(), 'best_model.pt') # save best model

   # print('Epoch: %d, Validation Loss: %.3f' % (epoch+1, total_val_loss/len(valid_loader)))

# Test
total_test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs, labels = data
        outputs = model(inputs)
        test_loss = criterion(outputs, labels)
        total_test_loss += test_loss.item()
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()        
#print('Totals : Test Loss: %.3f, Test Accuracy: %.3f' % (total_test_loss/len(test_loader), correct/total))


best_model = MLP(input_size=13)
best_model.load_state_dict(torch.load("best_model.pt"))

# Test veri setini yükleyin
test_dataset = CustomDataset('cure_the_princess_test.csv')
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Test veri seti üzerinde modelin performansını ölçün
with torch.no_grad():
    true_labels = []
    predicted_labels = []
    for i, data in enumerate(test_loader):
        inputs, labels = data
        outputs = best_model(inputs)
        predicted = torch.round(outputs)
        true_labels += labels.squeeze().tolist()
        predicted_labels += predicted.squeeze().tolist()

accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)

print('Accuracy: %.3f, F1: %.3f, Precision: %.3f, Recall: %.3f' % (accuracy, f1, precision, recall))

Accuracy: 0.931, F1: 0.928, Precision: 0.980, Recall: 0.881
